## Create Two Tables: People and Assignments

In [95]:
cursor = db.final_profile_details.find({}, {"assignments": 1, "ciphertext": 1})

In [96]:
cursor.count()

97455

In [12]:
import pandas as pd

In [97]:
df =pd.DataFrame(list(cursor))

In [98]:
df.columns

Index([u'_id', u'assignments', u'ciphertext'], dtype='object')

## Assignments 

In [4]:
import pandas as pd

In [5]:
cursor = db.final_profile_details.find({}, {"assignments": 1, "ciphertext": 1})
df =  pd.DataFrame(list(cursor))

Separate fixed and hourly assignments

In [6]:
fixed_assignments = df["assignments"].apply(lambda x: x["fp"])
hourly_assignments = df["assignments"].apply(lambda x: x["hr"])
df["hourly_assignments"] = hourly_assignments
df["fixed_assignments"] = fixed_assignments

In [7]:
h_assignments = pd.DataFrame()
h_assignments["assignment"] = hourly_assignments
h_assignments["ciphertext"] = df["ciphertext"]
h_assignments = h_assignments[h_assignments["assignment"] != ""]

In [8]:
f_assignments = pd.DataFrame()
f_assignments["assignment"] = fixed_assignments
f_assignments["ciphertext"] = df["ciphertext"]
f_assignments = f_assignments[f_assignments["assignment"] != ""]

Unnest assignments

In [9]:
def unnest_anything(df, target, sub_target):  
    # initialize new dataset
    nd = pd.DataFrame.from_dict(df[target][1])
    nd["ciphertext"] = "lksjdflkj"
    for index, row in df.iterrows():
        # if the person has it
        if row[target]: 
            # if person has multiple, put each in own row
            if type(row[target][sub_target]) == list: 
                td = pd.DataFrame.from_dict(row[target])
                td["ciphertext"] = row["ciphertext"]  
                nd = pd.concat([nd, td], ignore_index=True)
            # if person has one, keep the same
            else: 
                td = pd.DataFrame(index = [0], columns = [sub_target, "ciphertext"])
                td = td.set_value(0, 'ciphertext', row["ciphertext"])
                td = td.set_value(0, sub_target, row[target][sub_target])
                nd = pd.concat([nd, td], ignore_index=True)
    return nd

In [10]:
hourly_partially_unnested = unnest_anything(h_assignments, "assignment", "job")

In [124]:
def finish_unnesting_assignments(df, target):      
    new_df = pd.DataFrame()
    for key in df[target].iloc[11].keys(): 
        key_values = df[target].apply(lambda x: x.get(key))
        new_df[key] = key_values
    new_df["ciphertext"] = df["ciphertext"]
    return new_df

In [125]:
hourly_completely_unnested = finish_unnesting_assignments(hourly_partially_unnested, "job")
fixed_partially_unnested = unnest_anything(f_assignments, "assignment", "job")
fixed_completely_unnested = finish_unnesting_assignments(fixed_partially_unnested, "job")

In [128]:
assignments_unnested = pd.concat([fixed_completely_unnested, hourly_completely_unnested])
assignments_unnested = assignments_unnested[assignments_unnested["ciphertext"] != "lksjdflkj"]
len(assignments_unnested)

249092

In [66]:
import pickle

In [172]:
# drop any duplicates
assignments_unnested = pd.DataFrame.drop_duplicates(assignments_unnested)

In [174]:
with open('assignments.pkl', 'w') as picklefile:
    pickle.dump(assignments_unnested, picklefile)

### Check how many jobs don't have payment rate

What percent of fixed assignments don't have a total charge?

In [117]:
float(pd.isnull(fixed_completely_unnested["as_total_charge"]).sum())/len(fixed_completely_unnested)

0.4132832721362788

About 41% don't have a total charge

How many hourly don't have a rate?

In [130]:
hourly_completely_unnested.head()

,as_to,as_rate,as_financial_privacy,as_total_hours,as_from_full,as_ciphertext_opening_recno,as_total_hours_precise,as_opening_title,as_job_type,as_to_full,as_agency_name,as_total_charge,ciphertext
0,08/2016,None,None,3.00,None,None,3,Part2,Hourly,08/19/2016,None,None,lksjdflkj
1,07/2016,$40.00,None,3.67,None,None,3.66666667,Develop a model for providing a quantitative e...,Hourly,07/07/2016,None,40,lksjdflkj
2,08/2016,None,None,3.00,None,None,3,Part2,Hourly,08/19/2016,None,None,~01d3ad50155896e4ae
3,07/2016,$40.00,None,3.67,None,None,3.66666667,Develop a model for providing a quantitative e...,Hourly,07/07/2016,None,40,~01d3ad50155896e4ae
4,08/2016,None,None,3.00,None,None,3,Part2,Hourly,08/19/2016,None,None,~01d3ad50155896e4ae


In [131]:
float(pd.isnull(hourly_completely_unnested["as_rate"]).sum())/len(hourly_completely_unnested)

0.4262280973527488

About 42% don't have a rate

## Getting Profile Dataset

In [80]:
ids_with_jobs = assignments_unnested["ciphertext"].unique()

In [75]:
import numpy as np

In [82]:
ids_with_jobs = ids_with_jobs.tolist()

In [84]:
cursor = db.final_profile_details.find({"ciphertext": { "$in": ids_with_jobs}}, {"assignments": 0, "dev_last_worked_ts": 0,
         "dev_portrait": 0, "dev_portrait_100": 0, "dev_portrait_32": 0, "dev_portrait_50": 0,
          "dev_timezone": 0, "permalink": 0, "ag_cny_recno":0, "ag_country":0, "ag_country_tz":0,
           "ag_description": 0, "ag_logo": 0, "ag_name": 0, "ag_recent_hours": 0, 
           "ag_total_hours": 0})

In [85]:
cursor.count()

21603

We're only interested right now in people who ever completed a job. 

In [86]:
person_df =  pd.DataFrame(list(cursor))

In [150]:
person_df.head()

,_id,agency_ciphertext,ciphertext,dev_ac_agencies,dev_adj_score,dev_adj_score_recent,dev_bill_rate,dev_billed_assignments,dev_blurb,dev_city,...,dev_tot_feedback,dev_total_hours,dev_ui_profile_access,education,experiences,job_categories,portfolio_items,portfolio_items_v2,skills,tsexams
0,57c488aa97f6b341e8728c4d,NaN,~01d3ad50155896e4ae,,5,5,35.00,8,Data Science Consultant specialising in statis...,Cork,...,6,6.66666667,oDesk Users only,"{u'institution': [{u'ed_to': u'01/2014', u'ed_...","{u'experience': [{u'exp_from': u'09/2015', u'e...",NaN,,NaN,"{u'skill': [{u'skl_name': u'econometrics', u's...","{u'tsexam': [{u'ts_percentile': u'93', u'ts_du..."
1,57c4890097f6b341e8728c4e,NaN,~01d3ad50155896e4ae,,5,5,35.00,8,Data Science Consultant specialising in statis...,Cork,...,6,6.66666667,oDesk Users only,"{u'institution': [{u'ed_to': u'01/2014', u'ed_...","{u'experience': [{u'exp_from': u'09/2015', u'e...",NaN,,NaN,"{u'skill': [{u'skl_name': u'econometrics', u's...","{u'tsexam': [{u'ts_percentile': u'93', u'ts_du..."
2,57c4890297f6b341e8728c51,NaN,~0199f1742b3c7202ba,,5,5,18.00,2,"TECHNICAL SKILLS\nMicrosoft Office: Word, Exce...",Tampa,...,2,91.50000001,Public,"{u'institution': [{u'ed_to': u'01/2016', u'ed_...","{u'experience': {u'exp_from': u'06/2011', u'ex...",NaN,,{u'portfolio_items_v': [{u'pi_large_thumbnail'...,"{u'skill': [{u'skl_name': u'data-analysis', u'...","{u'tsexam': [{u'ts_percentile': u'84', u'ts_du..."
3,57c4890397f6b341e8728c52,NaN,~010ee721f96dbeb744,,4.9733793523,5,8.00,32,"Talented, Well qualified and versatile profess...",Bangalore,...,20,422.4,Public,"{u'institution': {u'ed_to': u'01/2012', u'ed_a...","{u'experience': {u'exp_from': u'03/2013', u'ex...",NaN,{u'portfolio_item': [{u'pi_large_thumbnail': u...,{u'portfolio_items_v': [{u'pi_large_thumbnail'...,"{u'skill': [{u'skl_name': u'article-writing', ...","{u'tsexam': [{u'ts_percentile': u'65', u'ts_du..."
4,57c4890397f6b341e8728c53,NaN,~0113b7a1ad10ba8634,,5,5,25.00,1,I graduated this spring with a Master's in Mat...,Houston,...,1,7.33333333,Public,"{u'institution': [{u'ed_to': u'01/2016', u'ed_...","{u'experience': [{u'exp_from': u'05/2015', u'e...",NaN,,NaN,"{u'skill': [{u'skl_name': u'apache-spark', u's...",


In [153]:
person_df_lim = person_df.drop(["skills", "tsexams", "experiences", "education"], axis = 1)

In [198]:
pd.set_option('max_columns',50)

In [199]:
person_df_lim.head()

,_id,agency_ciphertext,ciphertext,dev_ac_agencies,dev_adj_score,dev_adj_score_recent,dev_bill_rate,dev_billed_assignments,dev_blurb,dev_city,dev_country,dev_eng_skill,dev_first_name,dev_groups,dev_is_affiliated,dev_job_categories_v2,dev_last_activity,dev_last_name,dev_last_worked,dev_portfolio_items_count,dev_portfolio_items_v2_count,dev_profile_title,dev_recno,dev_recno_ciphertext,dev_short_name,dev_tot_feedback,dev_total_hours,dev_ui_profile_access,job_categories,portfolio_items,portfolio_items_v2
0,57c488aa97f6b341e8728c4d,NaN,~01d3ad50155896e4ae,,5,5,35.00,8,Data Science Consultant specialising in statis...,Cork,Ireland,5,Michael,,0,{u'dev_job_categories_v': [{u'name': u'Data Vi...,"August 15, 2016",Grogan,"August 15, 2016",0,0,Data Science and Statistics Consultant | Pytho...,7766913,~01d3ad50155896e4ae,Michael Grogan,6,6.66666667,oDesk Users only,NaN,,NaN
2,57c4890297f6b341e8728c51,NaN,~0199f1742b3c7202ba,,5,5,18.00,2,"TECHNICAL SKILLS\nMicrosoft Office: Word, Exce...",Tampa,United States,,Thu,,0,{u'dev_job_categories_v': [{u'name': u'Project...,"August 2, 2016",Nguyen,"June 12, 2016",0,2,Data Science Analytic,11492831,~0199f1742b3c7202ba,Thu Nguyen,2,91.50000001,Public,NaN,,{u'portfolio_items_v': [{u'pi_large_thumbnail'...
3,57c4890397f6b341e8728c52,NaN,~010ee721f96dbeb744,,4.9733793523,5,8.00,32,"Talented, Well qualified and versatile profess...",Bangalore,India,5,Keerthi,,0,{u'dev_job_categories_v': [{u'name': u'Present...,"August 29, 2016",,"August 29, 2016",5,5,"Analytics Consulting, Data Science",3844505,~010ee721f96dbeb744,Keerthi E.,20,422.4,Public,NaN,{u'portfolio_item': [{u'pi_large_thumbnail': u...,{u'portfolio_items_v': [{u'pi_large_thumbnail'...
4,57c4890397f6b341e8728c53,NaN,~0113b7a1ad10ba8634,,5,5,25.00,1,I graduated this spring with a Master's in Mat...,Houston,United States,5,Sarah,,0,{u'dev_job_categories_v': [{u'name': u'Data Vi...,"August 4, 2016",Armstrong,"August 4, 2016",0,0,Data science & statistical computing,9383018,~0113b7a1ad10ba8634,Sarah Armstrong,1,7.33333333,Public,NaN,,NaN
5,57c4890497f6b341e8728c54,NaN,~018a25773b7e402c48,,0,0,10.00,1,I have masters degree in computer science (UFP...,Belém,Brazil,,Leandro H S,,0,{u'dev_job_categories_v': [{u'name': u'Other -...,"August 28, 2016",Correa,"August 21, 2016",0,0,Data science/analytics,11761748,~018a25773b7e402c48,Leandro H S Correa,0,1.16666667,Public,NaN,,NaN


In [191]:
# drop any duplicates
person_df_lim = person_df_lim.drop_duplicates(["ciphertext"])

In [193]:
with open('profiles.pkl', 'w') as picklefile:
    pickle.dump(person_df_lim, picklefile)

I need to unpack education, experiences, skills, and tsexams

### Experiences

In [469]:
experiences_nested = person_df[["experiences", "ciphertext"]]

In [553]:
def unnest_experiences(df):  
    # initialize new dataset
    nd = pd.DataFrame.from_dict(df["experiences"][1])
    nd["ciphertext"] = "lksjdflkj"
    # for each row of the dataframe
    for index, row in df.iterrows():
        # if the person has experience
        if row["experiences"]: 
            # if person has multiple experiences, put each in own row
            if type(row["experiences"]["experience"]) == list: 
                td = pd.DataFrame.from_dict(row["experiences"])
                td["ciphertext"] = row["ciphertext"]  
                nd = pd.concat([nd, td], ignore_index=True)
            # if person has one experience, keep the same
            else: 
                td = pd.DataFrame(index = [0], columns = ["experience", "ciphertext"])
                td = td.set_value(0, 'ciphertext', row["ciphertext"])
                td = td.set_value(0, "experience", row["experiences"]["experience"])
                nd = pd.concat([nd, td], ignore_index=True)
    return nd

def finish_unnest_experiences(df):  
    # initialize new dataset
    nd = pd.DataFrame(df["experience"][0], index = [0])
    nd["ciphertext"] = "lksjdflkj"
    for index, row in df.iterrows():
        td = pd.DataFrame(row["experience"], index = [1])
        td["ciphertext"] = row["ciphertext"]  
        nd = pd.concat([nd, td], ignore_index = True)
    return nd

In [554]:
partially_unnested_experiences = unnest_experiences(experiences_nested)

In [577]:
completely_unnested_experiences = finish_unnest_experiences(partially_unnested_experiences)
completely_unnested_experiences = completely_unnested_experiences[completely_unnested_experiences["ciphertext"] != "lksjdflkj"]

In [582]:
completely_unnested_experiences.head()

,exp_comment,exp_company,exp_from,exp_title_raw,exp_to,ciphertext
3,,Nova School of Business and Economics,09/2015,Teaching Assistant (MSc Economics),02/2016,~01d3ad50155896e4ae
4,,Citco Fund Services,05/2013,Hedge Fund Administrator,02/2014,~01d3ad50155896e4ae
5,,Nova School of Business and Economics,09/2015,Teaching Assistant (MSc Economics),02/2016,~01d3ad50155896e4ae
6,,Citco Fund Services,05/2013,Hedge Fund Administrator,02/2014,~01d3ad50155896e4ae
7,- Founded a freelance marketing group that wor...,Self-employee,06/2011,Marketing,01/2013,~0199f1742b3c7202ba


In [160]:
# drop any duplicates
completely_unnested_experiences = pd.DataFrame.drop_duplicates(completely_unnested_experiences)

In [164]:
with open('experiences.pkl', 'w') as picklefile:
    pickle.dump(completely_unnested_experiences, picklefile)

## Skills

In [134]:
skills_nested = person_df[["skills", "ciphertext"]]

In [587]:
skills_nested["skills"][0]["skill"]

[{u'skl_description': u'Econometrics has been defined as the application of mathematics and statistical methods to economic data and described as the branch of economics that aims to give empirical content to economic relations.',
  u'skl_external_link': u'http://en.wikipedia.org/wiki/Econometrics',
  u'skl_has_tests': u'0',
  u'skl_name': u'econometrics',
  u'skl_rank': u'2'},
 {u'skl_description': u'Excel VBA (Visual Basic for Applications) is the programming language of Excel. Excel code processes are called Excel Macros.',
  u'skl_external_link': u'',
  u'skl_has_tests': u'0',
  u'skl_name': u'excel-vba',
  u'skl_rank': u'10'},
 {u'skl_description': u'Financial analysis (also referred to as financial statement analysis or accounting analysis) refers to an assessment of the viability, stability and profitability of a business, sub-business or project.',
  u'skl_external_link': u'http://en.wikipedia.org/wiki/Financial_analysis',
  u'skl_has_tests': u'1',
  u'skl_name': u'financial-an

In [211]:
def unnest_anything(df, target, sub_target):  
    # initialize new dataset
    nd = pd.DataFrame.from_dict(df[target].iloc[1])
    nd["ciphertext"] = "lksjdflkj"
    for index, row in df.iterrows():
        # if the person has it
        if row[target]: 
            # if person has multiple, put each in own row
            if type(row[target][sub_target]) == list: 
                td = pd.DataFrame.from_dict(row[target])
                td["ciphertext"] = row["ciphertext"]  
                nd = pd.concat([nd, td], ignore_index=True)
            # if person has one, keep the same
            else: 
                td = pd.DataFrame(index = [0], columns = [sub_target, "ciphertext"])
                td = td.set_value(0, 'ciphertext', row["ciphertext"])
                td = td.set_value(0, sub_target, row[target][sub_target])
                nd = pd.concat([nd, td], ignore_index=True)
    return nd

def finish_unnesting(df, target):      
    new_df = pd.DataFrame()
    for key in df[target].iloc[0].keys(): 
        key_values = df[target].apply(lambda x: x[key])
        new_df[key] = key_values
    new_df["ciphertext"] = df["ciphertext"]
    return new_df

In [136]:
len(skills_partially_unnested)

157709

In [145]:
skills_partially_unnested = unnest_anything(skills_nested, "skills", "skill")
skills_completely_unnested = pd.DataFrame()
skills_completely_unnested["ciphertext"] = skills_partially_unnested["ciphertext"]
skills_completely_unnested["skill_name"] = skills_partially_unnested["skill"].apply(lambda x: x["skl_name"])
skills_completely_unnested["skill_rank"] = skills_partially_unnested["skill"].apply(lambda x: x["skl_rank"])
skills_completely_unnested = skills_completely_unnested[skills_completely_unnested["ciphertext"] != "lksjdflkj"]
# drop any duplicates
skills_completely_unnested = pd.DataFrame.drop_duplicates(skills_completely_unnested)

In [166]:
with open('skills.pkl', 'w') as picklefile:
    pickle.dump(skills_completely_unnested, picklefile)

### Get skills for everyone

In [274]:
cursor = db.final_profile_details.find({}, {"ciphertext": 1, "skills": 1})

In [276]:
all_skills = pd.DataFrame(list(cursor))
all_skills_partially_unnested = unnest_anything(all_skills, "skills", "skill")
all_skills_completely_unnested = pd.DataFrame()
all_skills_completely_unnested["ciphertext"] = all_skills_partially_unnested["ciphertext"]
all_skills_completely_unnested["skill_name"] = all_skills_partially_unnested["skill"].apply(lambda x: x["skl_name"])
all_skills_completely_unnested["skill_rank"] = all_skills_partially_unnested["skill"].apply(lambda x: x["skl_rank"])
# drop any duplicates and the nonsense one
all_skills_completely_unnested = all_skills_completely_unnested[all_skills_completely_unnested["ciphertext"] != "lksjdflkj"]
all_skills_completely_unnested = pd.DataFrame.drop_duplicates(all_skills_completely_unnested)

In [283]:
with open('all_skills.pkl', 'w') as picklefile:
    pickle.dump(all_skills_completely_unnested, picklefile)

In [282]:
pd.DataFrame.to_csv(all_skills_completely_unnested, "all_skills.csv")

## Education

In [88]:
education_nested = person_df[["education", "ciphertext"]]
education_partially_unnested = unnest_anything(education_nested, "education", "institution")
education_completely_unnested = finish_unnesting(education_partially_unnested, "institution")
education_completely_unnested = education_completely_unnested[education_completely_unnested["ciphertext"] != "lksjdflkj"]
# drop any duplicates
education_completely_unnested = pd.DataFrame.drop_duplicates(education_completely_unnested)

In [168]:
with open('education.pkl', 'w') as picklefile:
    pickle.dump(education_completely_unnested, picklefile)

Education for everyone

In [284]:
cursor = db.final_profile_details.find({}, {"ciphertext": 1, "education": 1})
all_education = pd.DataFrame(list(cursor))

In [286]:
all_education_partially_unnested = unnest_anything(all_education, "education", "institution")
all_education_completely_unnested = finish_unnesting(all_education_partially_unnested, "institution")
all_education_completely_unnested = all_education_completely_unnested[all_education_completely_unnested["ciphertext"] != "lksjdflkj"]
# drop any duplicates
all_education_completely_unnested = pd.DataFrame.drop_duplicates(all_education_completely_unnested)

In [288]:
with open('all_education.pkl', 'w') as picklefile:
    pickle.dump(all_education_completely_unnested, picklefile)

### Exams

In [98]:
exam_nested = person_df[["tsexams", "ciphertext"]]
exam_partially_unnested = unnest_anything(exam_nested, "tsexams", "tsexam")
exam_completely_unnested = finish_unnesting(exam_partially_unnested, "tsexam")
exam_completely_unnested = exam_completely_unnested[exam_completely_unnested["ciphertext"] != "lksjdflkj"]
# drop any duplicates
exam_completely_unnested = pd.DataFrame.drop_duplicates(exam_completely_unnested)

In [170]:
with open('exams.pkl', 'w') as picklefile:
    pickle.dump(exam_completely_unnested, picklefile)